In [39]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from textblob import TextBlob
import nltk
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 

## training the model using UCI OnlineNewsPopularity data

In [9]:
table=pd.read_csv("OnlineNewsPopularity.csv")
table.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [12]:
table1=table.copy()

In [13]:
table1.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


### since this data does not have any misssing value procedding with cleaning 

In [15]:
#Cleaned the column names by removing white spaces and converting into lowercase.
clean_col={x:x.lower().strip() for x in list(table1)}
table1.rename(index=str, columns=clean_col,inplace=True)

In [ ]:
NON_PREDICTIVE_COLS = ['url', 'timedelta', 'shares']
TARGET_COL = ['shares']

In [33]:
# dropping all irrelevent columns
table1 = table1.drop(['url','timedelta', 'lda_00','lda_01','lda_02','lda_03','lda_04','num_self_hrefs','num_videos', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','rate_positive_words','rate_negative_words','abs_title_subjectivity','abs_title_sentiment_polarity','data_channel_is_lifestyle','data_channel_is_entertainment','data_channel_is_bus','data_channel_is_socmed','data_channel_is_tech','data_channel_is_world','weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday','weekday_is_sunday','is_weekend','num_imgs','num_hrefs','n_unique_tokens','n_non_stop_words','n_non_stop_unique_tokens','average_token_length','num_keywords','global_rate_positive_words','global_rate_negative_words','avg_positive_polarity','min_positive_polarity','max_positive_polarity','max_negative_polarity','min_negative_polarity','avg_negative_polarity'], axis=1)
print(table1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 39644 entries, 0 to 39643
Data columns (total 7 columns):
n_tokens_title               39644 non-null float64
n_tokens_content             39644 non-null float64
global_subjectivity          39644 non-null float64
global_sentiment_polarity    39644 non-null float64
title_subjectivity           39644 non-null float64
title_sentiment_polarity     39644 non-null float64
shares                       39644 non-null int64
dtypes: float64(6), int64(1)
memory usage: 2.4+ MB
None


In [35]:
X = table1[[x for x in list(table1) if x not in NON_PREDICTIVE_COLS]]
Y = table1[TARGET_COL]

In [36]:
Y['shares'] = Y['shares'].map(lambda x: 1 if x >= 1400 else 0) 
# taking the thresh as 1400 above it are popular and below this value are non-popular

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, random_state=1)

### TRAINING THE MODEL

In [107]:
#RandomForestRegressor
model = RandomForestRegressor() 
model.fit(X_test,y_test)  
y_pred = model.predict(X_test) 

accu = model.score(X_test,y_test)
print("Accuracy of RandomForest ",accu*100,'%')


E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy of RandomForest  84.58620730689437 %


### crawling the news

In [108]:
req = requests.get("https://timesofindia.indiatimes.com/world")

In [109]:
soup = BeautifulSoup(req.content, 'html.parser') 
table = soup.findAll('a',attrs = {'class':'w_img'}) 

In [110]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [111]:
data=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data1={}
    data1['Title']=article.title
    data1['Text']=article.text
    data1['Summary']=article.summary
    data1['Keywords']=article.keywords
    data.append(data1)

In [112]:
df=pd.DataFrame(data)
df.head()

,Keywords,Summary,Text,Title
0,"[children, spain, 331, virus, 24, counts, coro...",Families with their children walk along a boul...,Families with their children walk along a boul...,Spain Coronavirus: Spain counts 331 new virus ...
1,"[facebook, mayor, true, entertains, try, impos...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",Covid-19: Mayor entertains residents after imp...
2,"[recovering, virus, uk, worsened, worst, retur...","Apr 27, 2020, 08:40AM ISTSource: Times NowBrit...","Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",UK PM Boris Johnson returns to work after reco...
3,"[veteran, dead, say, law, shooting, officers, ...","Apr 27, 2020, 08:28AM ISTSource: APPolice say ...","Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...","1 officer dead, 1 wounded in Louisiana shooting"
4,"[turning, visibility, fresh, 22, transport, co...","Apr 22, 2020, 04:03PM ISTSource: APIndia's ext...","Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",Covid-19: Lockdown reveals fresh and clean air...


In [94]:
df['title_sentiment_polarity'] = df.apply(lambda x: TextBlob(x['Title']).sentiment.polarity, axis=1)

df['global_sentiment_polarity'] = df.apply(lambda x: TextBlob(x['Text']).sentiment.polarity, axis=1)

In [95]:
df['title_subjectivity'] = df.apply(lambda x: TextBlob(x['Title']).sentiment.subjectivity, axis=1)
df['global_subjectivity'] = df.apply(lambda x: TextBlob(x['Text']).sentiment.subjectivity, axis=1)
df.head()

,Keywords,Summary,Text,Title,title_sentiment_polarity,global_sentiment_polarity,title_subjectivity,global_subjectivity
0,"[children, spain, 331, virus, 24, counts, coro...",Families with their children walk along a boul...,Families with their children walk along a boul...,Spain Coronavirus: Spain counts 331 new virus ...,0.136364,0.235795,0.454545,0.335985
1,"[facebook, mayor, true, entertains, try, impos...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",Covid-19: Mayor entertains residents after imp...,0.000000,0.300000,0.000000,0.643750
2,"[recovering, virus, uk, worsened, worst, retur...","Apr 27, 2020, 08:40AM ISTSource: Times NowBrit...","Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",UK PM Boris Johnson returns to work after reco...,0.000000,-0.153125,0.000000,0.415625
3,"[veteran, dead, say, law, shooting, officers, ...","Apr 27, 2020, 08:28AM ISTSource: APPolice say ...","Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...","1 officer dead, 1 wounded in Louisiana shooting",-0.200000,0.028571,0.400000,0.085714
4,"[turning, visibility, fresh, 22, transport, co...","Apr 22, 2020, 04:03PM ISTSource: APIndia's ext...","Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",Covid-19: Lockdown reveals fresh and clean air...,0.333333,0.108939,0.600000,0.534242


In [96]:
title = df['Title'].str.split().str.len()
df['n_tokens_title'] = title

In [97]:
#number of words in Article
article = df['Text'].str.split().str.len()
df['n_tokens_content'] = article

In [102]:
final_data = pd.DataFrame(df)
final_data_test=final_data.drop(['Keywords','Summary','Text','Title'],axis=1)


In [105]:
final=pd.DataFrame(model.predict(final_data_test),final_data['Title'])
final.reset_index(level=0, inplace=True)
final = final.rename(columns={ 0: "Shares"})




In [115]:
final.nlargest(10,'Shares')# top 10 news which are more likely to get viral

,Title,Shares
11,U.S. House Speaker Nancy Pelosi endorses Joe B...,0.580000
20,Indian national killed in road accident in Sin...,0.580000
28,WhatsApp group helps airlift terminally-ill te...,0.572857
16,China targets US coronavirus response,0.570000
31,Covid-19: PM Modi interacts with CMs via video...,0.570000
14,"Boris Johnson back at work, warns against rela...",0.550000
21,Indian-American lawyer Seema Nanda to step dow...,0.550000
22,Indian-American woman elected to American Acad...,0.550000
27,Indian-American Congressman Ro Khanna in White...,0.550000
2,UK PM Boris Johnson returns to work after reco...,0.530000


## used UCI data to train the model  and modefying the scraped data for prediction 